In [2]:
import numpy as np
import pickle as pickle
from sklearn.preprocessing import normalize
import os
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, mean_absolute_error
from scipy.signal import welch
import os

In [3]:
channel = [1,2,3,4,6,11,13,17,19,20,21,25,29,31] #14 Channels chosen to fit Emotiv Epoch+
band = [4,8,12,16,25,45] #5 bands
window_size = 256 #Averaging band power of 2 sec
step_size = 16 #Each 0.125 sec update once
sample_rate = 128 #Sampling rate of 128 Hz
subjectList = ['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25','26','27','28','29','30','31','32']


In [3]:
def band_power(data, band, sample_rate):
    """Calculate band power for a given signal."""
    freqs, psd = welch(data, sample_rate, nperseg=sample_rate*2)
    bandpower = []
    for (low, high) in zip(band[:-1], band[1:]):
        idx_band = np.logical_and(freqs >= low, freqs < high)
        bandpower.append(np.sum(psd[idx_band]))
    return bandpower

def FFT_Processing(sub, channel, band, window_size, step_size, sample_rate):
    '''
    arguments:  string subject
                list channel indices
                list band
                int window size for FFT
                int step size for FFT
                int sample rate for FFT
    return:     void
    '''
    meta = []
    with open(f'data_preprocessed_python/s{sub}.dat', 'rb') as file:
        subject = pickle.load(file, encoding='latin1')  

        for i in range(40): 
            data = subject["data"][i]
            labels = subject["labels"][i]
            start = 0

            while start + window_size < data.shape[1]:
                meta_array = []
                meta_data = []  
                for j in channel:
                    X = data[j][start: start + window_size] 
                    Y = band_power(X, band, sample_rate)  
                    meta_data = meta_data + list(Y)

                meta_array.append(np.array(meta_data))
                meta_array.append(labels)

                meta.append(np.array(meta_array))    
                start += step_size
                
        meta = np.array(meta)

        if not os.path.exists('out'):
            os.makedirs('out')

        np.save(f'out/s{sub}', meta, allow_pickle=True, fix_imports=True)



In [ ]:
for subjects in subjectList:
    FFT_Processing(subjects, channel, band, window_size, step_size, sample_rate)

In [ ]:
import os
import numpy as np



data_training = []
label_training = []
data_testing = []
label_testing = []
data_validation = []
label_validation = []

for subjects in subjectList:
    file_path = os.path.join('out', f's{subjects}.npy')
    
    with open(file_path, 'rb') as file:
        sub = np.load(file, allow_pickle=True)
        for i in range(sub.shape[0]):
            if i % 8 == 0:
                data_testing.append(sub[i][0])
                label_testing.append(sub[i][1])
            elif i % 8 == 1:
                data_validation.append(sub[i][0])
                label_validation.append(sub[i][1])
            else:
                data_training.append(sub[i][0])
                label_training.append(sub[i][1])





np.save(os.path.join('out', 'data_training'), np.array(data_training), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_training'), np.array(label_training), allow_pickle=True, fix_imports=True)
print("training dataset:", np.array(data_training).shape, np.array(label_training).shape)

np.save(os.path.join('out', 'data_testing'), np.array(data_testing), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_testing'), np.array(label_testing), allow_pickle=True, fix_imports=True)
print("testing dataset:", np.array(data_testing).shape, np.array(label_testing).shape)

np.save(os.path.join('out', 'data_validation'), np.array(data_validation), allow_pickle=True, fix_imports=True)
np.save(os.path.join('out', 'label_validation'), np.array(label_validation), allow_pickle=True, fix_imports=True)
print("validation dataset:", np.array(data_validation).shape, np.array(label_validation).shape)


In [ ]:
with open('out\data_training.npy', 'rb') as fileTrain:
    X  = np.load(fileTrain)
    
with open('out\label_training.npy', 'rb') as fileTrainL:
    Y  = np.load(fileTrainL)
    
X = normalize(X)
Z = np.ravel(Y[:, [1]])

Arousal_Train = np.ravel(Y[:, [0]])
Valence_Train = np.ravel(Y[:, [1]])
Dominance_Train = np.ravel(Y[:, [2]])
Liking_Train = np.ravel(Y[:, [3]])



with open('out\data_validation.npy', 'rb') as fileTrain:
    M  = np.load(fileTrain)
    
with open('out\label_validation.npy', 'rb') as fileTrainL:
    N  = np.load(fileTrainL)

M = normalize(M)
L = np.ravel(N[:, [1]])

Arousal_Test = np.ravel(N[:, [0]])
Valence_Test = np.ravel(N[:, [1]])
Dominance_Test = np.ravel(N[:, [2]])
Liking_Test = np.ravel(N[:, [3]])

In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=17, learning_rate=0.1, n_jobs=6)

xgb_reg.fit(X[0:468480:32], Arousal_Train[0:468480:32])


predictions = xgb_reg.predict(M[0:78080:32])


mse = mean_squared_error(Arousal_Test[0:78080:32], predictions)
print(f"Mean Squared Error: {mse}")


accuracy = accuracy_score((Arousal_Test[0:78080:32] > 5), (predictions > 5))
print(f"Classification Accuracy: {accuracy}")


std_dev = np.std(predictions)
print(f"Standard Deviation of Predictions: {std_dev}")

f1 = f1_score((Arousal_Test[0:78080:32] > 5), (predictions > 5))
print(f"F1 Score: {f1}")

mae = mean_absolute_error(Arousal_Test[0:78080:32], predictions)
print(f"Mean Absolute Error: {mae}")


In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=12, learning_rate=0.1, n_jobs=6)

xgb_reg.fit(X[0:468480:32], Valence_Train[0:468480:32])


predictions = xgb_reg.predict(M[0:78080:32])


mse = mean_squared_error(Valence_Test[0:78080:32], predictions)
print(f"Mean Squared Error: {mse}")


accuracy = accuracy_score((Valence_Test[0:78080:32] > 5), (predictions > 5))
print(f"Classification Accuracy: {accuracy}")


std_dev = np.std(predictions)
print(f"Standard Deviation of Predictions: {std_dev}")

f1 = f1_score((Valence_Test[0:78080:32] > 5), (predictions > 5))
print(f"F1 Score: {f1}")

mae = mean_absolute_error(Valence_Test[0:78080:32], predictions)
print(f"Mean Absolute Error: {mae}")


In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=12, learning_rate=0.1, n_jobs=6)

xgb_reg.fit(X[0:468480:32], Dominance_Train[0:468480:32])


predictions = xgb_reg.predict(M[0:78080:32])


mse = mean_squared_error(Dominance_Test[0:78080:32], predictions)
print(f"Mean Squared Error: {mse}")


accuracy = accuracy_score((Dominance_Test[0:78080:32] > 5), (predictions > 5))
print(f"Classification Accuracy: {accuracy}")


std_dev = np.std(predictions)
print(f"Standard Deviation of Predictions: {std_dev}")

f1 = f1_score((Dominance_Test[0:78080:32] > 5), (predictions > 5))
print(f"F1 Score: {f1}")

mae = mean_absolute_error(Dominance_Test[0:78080:32], predictions)
print(f"Mean Absolute Error: {mae}")


In [ ]:
xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=10, learning_rate=0.1, n_jobs=6)

xgb_reg.fit(X[0:468480:32], Liking_Train[0:468480:32])


predictions = xgb_reg.predict(M[0:78080:16])


mse = mean_squared_error(Liking_Test[0:78080:16], predictions)
print(f"Mean Squared Error: {mse}")


accuracy = accuracy_score((Liking_Test[0:78080:16] > 5), (predictions > 5))
print(f"Classification Accuracy: {accuracy}")


std_dev = np.std(predictions)
print(f"Standard Deviation of Predictions: {std_dev}")

f1 = f1_score((Liking_Test[0:78080:16] > 5), (predictions > 5))
print(f"F1 Score: {f1}")

mae = mean_absolute_error(Liking_Test[0:78080:16], predictions)
print(f"Mean Absolute Error: {mae}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score


# List of n_estimators to test
n_estimators_list = [100,300,500, 600, 700, 800]

# Loop through each n_estimators value
for n in n_estimators_list:
    # Initialize the model with the current n_estimators value
    xgb_reg = xgb.XGBRegressor(n_estimators=n, max_depth=12, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Liking_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate accuracy
    accuracy = accuracy_score((Liking_Test[0:78080:16] > 5), (predictions > 5))
    print(f"n_estimators: {n}, Classification Accuracy: {accuracy}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np

print(f"Shape of X: {X.shape}")
print(f"Shape of Valence_Train: {Valence_Train.shape}")
print(f"Shape of M: {M.shape}")
print(f"Shape of Valence_Test: {Valence_Test.shape}")

# List of n_estimators to test
n_estimators_list = [100, 300, 550, 650, 700,800]

# Loop through each n_estimators value
for n in n_estimators_list:
    # Initialize the model with the current n_estimators value
    xgb_reg = xgb.XGBRegressor(n_estimators=n, max_depth=12, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Valence_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate accuracy
    accuracy = accuracy_score((Valence_Test[0:78080:16] > 5), (predictions > 5))
    print(f"n_estimators: {n}, Classification Accuracy: {accuracy}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np

print(f"Shape of X: {X.shape}")
print(f"Shape of Dominance_Train: {Dominance_Train.shape}")
print(f"Shape of M: {M.shape}")
print(f"Shape of Dominance_Test: {Dominance_Test.shape}")

# List of n_estimators to test
n_estimators_list = [100,400,500, 600, 700, 800]

# Loop through each n_estimators value
for n in n_estimators_list:
    # Initialize the model with the current n_estimators value
    xgb_reg = xgb.XGBRegressor(n_estimators=n, max_depth=12, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Dominance_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate accuracy
    accuracy = accuracy_score((Dominance_Test[0:78080:16] > 5), (predictions > 5))
    print(f"n_estimators: {n}, Classification Accuracy: {accuracy}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np

print(f"Shape of X: {X.shape}")
print(f"Shape of Arousal_Train: {Arousal_Train.shape}")
print(f"Shape of M: {M.shape}")
print(f"Shape of Arousal_Test: {Arousal_Test.shape}")

# List of n_estimators to test
n_estimators_list = [100,400,500, 600, 700, 800]

# Loop through each n_estimators value
for n in n_estimators_list:
    # Initialize the model with the current n_estimators value
    xgb_reg = xgb.XGBRegressor(n_estimators=n, max_depth=12, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Arousal_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate accuracy
    accuracy = accuracy_score((Arousal_Test[0:78080:16] > 5), (predictions > 5))
    print(f"n_estimators: {n}, Classification Accuracy: {accuracy}")


In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, mean_absolute_error
import numpy as np


print(f"Shape of X: {X.shape}")
print(f"Shape of Liking_Train: {Liking_Train.shape}")
print(f"Shape of M: {M.shape}")
print(f"Shape of Liking_Test: {Liking_Test.shape}")

# List of max_depth values to test
max_depth_list = [6, 8, 10, 12, 15,17,19,20,21]

# Loop through each max_depth value
for max_depth in max_depth_list:
    # Initialize the model with the current max_depth value
    xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=max_depth, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Liking_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate metrics
    accuracy = accuracy_score((Liking_Test[0:78080:16] > 5), (predictions > 5))
    
    # Print the results
    print(f"Max Depth: {max_depth}")

    print(f"  Classification Accuracy: {accuracy}")



In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, accuracy_score, f1_score, mean_absolute_error
import numpy as np

# Example initialization of Valence_Train and Valence_Test

print(f"Shape of X: {X.shape}")
print(f"Shape of Valence_Train: {Valence_Train.shape}")
print(f"Shape of M: {M.shape}")
print(f"Shape of Valence_Test: {Valence_Test.shape}")

# List of max_depth values to test
max_depth_list = [ 10, 12, 15,16,17,18,19,20,21]

# Loop through each max_depth value
for max_depth in max_depth_list:
    # Initialize the model with the current max_depth value
    xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=max_depth, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Valence_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:16])
    
    # Calculate metrics
    accuracy = accuracy_score((Valence_Test[0:78080:16] > 5), (predictions > 5))

    # Print the results
    print(f"Max Depth: {max_depth}")

    print(f"  Classification Accuracy: {accuracy}")



In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
import numpy as np



# List of max_depth values to test
max_depth_list = [6, 8, 10, 12, 15,17,19,20,21]

# Loop through each max_depth value
for max_depth in max_depth_list:
    # Initialize the model with the current max_depth value
    xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=max_depth, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Arousal_Train[0:468480:32])
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:32])
    
    # Calculate accuracy
    accuracy = accuracy_score((Arousal_Test[0:78080:32] > 5), (predictions > 5))
    
    # Print the results
    print(f"Max Depth: {max_depth}")
    print(f"  Classification Accuracy: {accuracy}")
    print()


In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

# Assuming you have data arrays similar to Arousal_Train, Arousal_Test, and M for dominance

# List of max_depth values to test
max_depth_list = [6, 8, 10, 12, 15, 17, 19, 20, 21]

# Loop through each max_depth value
for max_depth in max_depth_list:
    # Initialize the model with the current max_depth value
    xgb_reg = xgb.XGBRegressor(n_estimators=800, max_depth=max_depth, learning_rate=0.1, n_jobs=6)
    
    # Train the model
    xgb_reg.fit(X[0:468480:32], Dominance_Train[0:468480:32])  # Assuming Dominance_Train is your training data for dominance
    
    # Make predictions
    predictions = xgb_reg.predict(M[0:78080:32])  # Assuming M is your test data for dominance
    
    # Since this is regression, you might want to threshold predictions for classification purposes
    # Example: Assuming dominance values > 5 are considered high dominance
    dominance_predictions = (predictions > 5)
    
    # Calculate accuracy
    accuracy = accuracy_score((Dominance_Test[0:78080:32] > 5), dominance_predictions)
    
    # Print the results
    print(f"Max Depth: {max_depth}")
    print(f"  Classification Accuracy: {accuracy}")
    print()
